## Metrics calculation

Based on attentions, tf-idf, etc...

### 1) Configuration

In [1]:
import pandas as pd
import numpy as np
import nltk
import string

STOPWORDS = nltk.corpus.stopwords.words('portuguese')

FOLDER = './outputs/DS3/bert-base-multilingual-cased'

COUNTS = {}
for i, row in pd.read_csv(f'{FOLDER}/result_counts.csv').iterrows():
    COUNTS[row['name']] = row['count']
    
token_metrics = []
COUNTS

{'antivacina_correct': 497,
 'antivacina_wrong': 94,
 'antivacina': 591,
 'provacina_correct': 506,
 'provacina_wrong': 103,
 'provacina': 609,
 'total': 1200}

### 2) Words metrics

In [2]:
words_metrics = {}

#### Attention (General and by group)

In [3]:
words_attention_df = pd.read_csv(f'{FOLDER}/words_attention.csv', sep=';')
words_attention_df['tokens'] = words_attention_df.tokens.apply(lambda x: str(x).split(' '))
words_attention_df = words_attention_df.loc[
    (words_attention_df.word.str.len() > 1) &\
    (~words_attention_df.word.isin(STOPWORDS)) &\
    (~words_attention_df.word.isin(list(string.punctuation)))
] # Remove words in stopwords
words_attention_df

,word,tokens,antivacina_correct_w,antivacina_correct_n,antivacina_incorrect_w,antivacina_incorrect_n,provacina_correct_w,provacina_correct_n,provacina_incorrect_w,provacina_incorrect_n
24,aa,[aa],0.00000,0,0.0,0,1.000000,1,0.000000,0
25,aaaaaaaaahhh,"[aa, ##aa, ##aa, ##aa, ##ah, ##h, ##h]",0.00000,0,0.0,0,0.353806,1,0.000000,0
26,aaaaamo,"[aa, ##aa, ##amo]",0.00000,0,0.0,0,0.000000,0,0.206031,1
27,aaai,"[aa, ##ai]",0.00000,0,0.0,0,0.367710,1,0.000000,0
28,abaixo,[abaixo],0.91885,2,0.0,0,0.000000,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...
4436,últimos,[últimos],0.00000,0,0.0,0,0.125319,1,0.000000,0
4437,única,[única],0.00000,0,0.0,0,0.216406,2,0.000000,0
4438,único,[único],0.00000,0,0.0,0,0.026346,1,0.000000,0
4439,útil,[útil],0.00000,0,0.0,0,0.189593,2,0.000000,0


In [4]:
for i, row in words_attention_df.iterrows():
    if row.word not in words_metrics:
        words_metrics[row.word] = {
            'absolute': 0.0,
            'absolute_correct': 0.0,
            'absolute_incorrect': 0.0,
            
#             'antichina_absolute': 0.0,
#             'antichina_absolute_correct': 0.0,
#             'antichina_absolute_incorrect': 0.0,
            
            'antivacina_absolute': 0.0,
            'antivacina_absolute_correct': 0.0,
            'antivacina_absolute_incorrect': 0.0,
            
            'provacina_absolute': 0.0,
            'provacina_absolute_correct': 0.0,
            'provacina_absolute_incorrect': 0.0,
            
            'tfidf': 0.0,
            
#             'antichina_tfidf': 0.0,
            'antivacina_tfidf': 0.0,
            'provacina_tfidf': 0.0,
            
#             'antichina_tfidf_correct': 0.0,
            'antivacina_tfidf_correct': 0.0,
            'provacina_tfidf_correct': 0.0,
            
#             'antichina_tfidf_incorrect': 0.0,
            'antivacina_tfidf_incorrect': 0.0,
            'provacina_tfidf_incorrect': 0.0,
        }
    
#     words_metrics[row.word]['absolute'] = (
#         (
#             (
#                 row.antichina_correct_w + row.antivacina_correct_w + row.provacina_correct_w +\
#                 row.antichina_incorrect_w + row.antivacina_incorrect_w + row.provacina_incorrect_w
#             ) / COUNTS['total'] # Maybe this should be the sum of ALL WEIGHTS.
#         ) * 100 
#     )
    words_metrics[row.word]['absolute'] = (
        (
            (
                row.antivacina_correct_w + row.provacina_correct_w +\
                row.antivacina_incorrect_w + row.provacina_incorrect_w
            ) / COUNTS['total'] # Maybe this should be the sum of ALL WEIGHTS.
        ) * 100 
    )
    
#     words_metrics[row.word]['absolute_correct'] = (
#         (
#             (
#                 row.antichina_correct_w + row.antivacina_correct_w + row.provacina_correct_w
#             ) / COUNTS['total'] # Maybe this should be the sum of ALL WEIGHTS.
#         ) * 100 
#     )
    words_metrics[row.word]['absolute_correct'] = (
        (
            (
                row.antivacina_correct_w + row.provacina_correct_w
            ) / COUNTS['total'] # Maybe this should be the sum of ALL WEIGHTS.
        ) * 100 
    )
    
#     words_metrics[row.word]['absolute_incorrect'] = (
#         (
#             (
#                 row.antichina_incorrect_w + row.antivacina_incorrect_w + row.provacina_incorrect_w
#             ) / COUNTS['total'] # Maybe this should be the sum of ALL WEIGHTS.
#         ) * 100 
#     )
    words_metrics[row.word]['absolute_incorrect'] = (
        (
            (
                row.antivacina_incorrect_w + row.provacina_incorrect_w
            ) / COUNTS['total'] # Maybe this should be the sum of ALL WEIGHTS.
        ) * 100 
    )
    
    
    # Now calculate for each group
    for group in [
#         'antichina',
        'antivacina',
        'provacina'
    ]:
        
        words_metrics[row.word][f'{group}_absolute'] = (
            (
                (
                    row[f'{group}_correct_w'] + row[f'{group}_incorrect_w']
                ) / COUNTS[group] # Maybe this should be the sum of ALL WEIGHTS.
            ) * 100 
        )
        
        words_metrics[row.word][f'{group}_absolute_correct'] = (
            (
                (
                    row[f'{group}_correct_w']
                ) / COUNTS[group] # Maybe this should be the sum of ALL WEIGHTS.
            ) * 100 
        )
        
        words_metrics[row.word][f'{group}_absolute_incorrect'] = (
            (
                (
                    row[f'{group}_incorrect_w']
                ) / COUNTS[group] # Maybe this should be the sum of ALL WEIGHTS.
            ) * 100 
        )
        
        if (row[f'{group}_correct_n'] + row[f'{group}_incorrect_n']) == 0:
            break
        
        words_metrics[row.word][f'{group}_relative'] = (
            (
                (
                    row[f'{group}_correct_w'] + row[f'{group}_incorrect_w']
                ) / (
                    row[f'{group}_correct_n'] + row[f'{group}_incorrect_n']
                )
            ) * 100 
        )

        words_metrics[row.word][f'{group}_relative_correct'] = (
            (
                (
                    row[f'{group}_correct_w']
                ) / (
                    row[f'{group}_correct_n'] + row[f'{group}_incorrect_n']
                )
            ) * 100 
        )

        words_metrics[row.word][f'{group}_relative_incorrect'] = (
            (
                (
                    row[f'{group}_incorrect_w']
                ) / (
                    row[f'{group}_correct_n'] + row[f'{group}_incorrect_n']
                )
            ) * 100 
        )

#### TF-IDF (General and by group)

In [5]:
tfidf_weights = pd.read_csv(f'{FOLDER}/tfidf_weights.csv', sep=';', index_col=0)
tfidf_weights

,0,1,2,3,4,5,6,7,8,9,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
aa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaaaaaaahhh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaaamo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abaixo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
única,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
único,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
útil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ลา,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
from ast import literal_eval

tfidf_indices = pd.read_csv(f'{FOLDER}/tfidf_indices.csv', sep=';', index_col=0)
# tfidf_indices.antichina = tfidf_indices.antichina.apply(lambda x: literal_eval(x))
tfidf_indices.antivacina = tfidf_indices.antivacina.apply(lambda x: literal_eval(x))
tfidf_indices.provacina = tfidf_indices.provacina.apply(lambda x: literal_eval(x))

tfidf_indices

,antivacina,provacina
group,,
all,"[0, 1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 14, 15, 1...","[4, 6, 13, 16, 21, 22, 24, 29, 31, 34, 35, 37,..."
correct,"[0, 1, 2, 3, 7, 8, 9, 10, 11, 14, 15, 17, 18, ...","[6, 13, 16, 21, 22, 24, 31, 37, 39, 40, 41, 42..."
wrong,"[5, 12, 20, 68, 75, 101, 111, 133, 146, 151, 1...","[4, 29, 34, 35, 38, 43, 47, 60, 69, 90, 93, 95..."


In [7]:
for _, row in tfidf_weights.iterrows():
    
    try:
    
        words_metrics[row.name]['tfidf'] = np.mean([row[col] for col in tfidf_weights.columns])

        for group in [
#             'antichina',
            'antivacina',
            'provacina'
        ]:

            group_tfidf_values = [row[index] for index in tfidf_indices[group][0]]

            words_metrics[row.name][f'{group}_tfidf'] = np.mean(
                group_tfidf_values
            )

            words_metrics[row.name][f'{group}_tfidf_correct'] = (
                np.sum([row[index] for index in tfidf_indices[group][1]]) / len(group_tfidf_values)
            )

            words_metrics[row.name][f'{group}_tfidf_incorrect'] = (
                np.sum([row[index] for index in tfidf_indices[group][2]]) / len(group_tfidf_values)
            )
    
    except:
        print('Error with word:', row.name)

Error with word: deliberdadepara
Error with word: mcdonalds
Error with word: ลา
Error with word: อบต


#### Save DataFrame

In [8]:
words_metrics_df = pd.DataFrame(words_metrics).T
words_metrics_df.insert(0, column='word', value=words_metrics_df.index)
words_metrics_df.to_csv(f'{FOLDER}/words_metrics.csv', index=None)
words_metrics_df.sample(5)

,word,absolute,absolute_correct,absolute_incorrect,relative,relative_correct,relative_incorrect,antivacina_absolute,antivacina_absolute_correct,antivacina_absolute_incorrect,...,provacina_relative,provacina_relative_correct,provacina_relative_incorrect,tfidf,antivacina_tfidf,provacina_tfidf,antivacina_tfidf_correct,provacina_tfidf_correct,antivacina_tfidf_incorrect,provacina_tfidf_incorrect
calados,calados,0.016915,0.016915,0.000000,20.298238,20.298238,0.000000,0.034346,0.034346,0.0,...,0.0,0.0,0.0,0.000365,0.000730,0.000000,0.00073,0.000000,0.000000,0.0
encherguei,encherguei,0.010609,0.000000,0.010609,12.730986,0.000000,12.730986,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000833,0.001667,0.000000,0.00000,0.000000,0.001667,0.0
caçamba,caçamba,0.005583,0.000000,0.005583,6.699087,0.000000,6.699087,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000349,0.000698,0.000000,0.00000,0.000000,0.000698,0.0
restrito,restrito,0.008056,0.008056,0.000000,9.667362,9.667362,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000197,0.000000,0.000395,0.00000,0.000395,0.000000,0.0
resiste,resiste,0.004815,0.004815,0.000000,5.777549,5.777549,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000331,0.000000,0.000662,0.00000,0.000662,0.000000,0.0
